In [1]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import random
from PIL import Image
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed

#make_subplots 
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt

import os
import sys
import json

from tqdm.notebook import tqdm

In [2]:
TEST_DATA_FOLDER = "./Test"
TRAIN_DATA_FOLDER = "./Train"

DESTINATION_FOLDER = "/Volumes/DATASET/"

BATCH_SIZE = 1000


In [10]:
def search_files_in_subdirectory(subdirectory, extension):
    files = []
    for root, _, filenames in os.walk(subdirectory):
        for filename in filenames:

            if filename.endswith(extension):
                files.append(os.path.join(root, filename))
            elif extension == "jpg":
                if filename.endswith("jpeg"):
                    files.append(os.path.join(root, filename))
                
    return files

def search_files_in_directory(directory,extension, num_threads=8):
    # Get the list of subdirectories
    subdirectories = [os.path.join(directory, d) for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]

    all_files = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = {executor.submit(search_files_in_subdirectory, subdirectory,extension): subdirectory for subdirectory in subdirectories}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Searching subdirectories"):
            all_files.extend(future.result())

    print(f"Found {len(all_files)} files in {directory} with extension {extension}")
    return all_files

In [4]:
all_test_files_json = search_files_in_directory(TEST_DATA_FOLDER, ".json")
all_train_files_json = search_files_in_directory(TRAIN_DATA_FOLDER, ".json")



Searching subdirectories:   0%|          | 0/2 [00:00<?, ?it/s]

Found 3234 files in ./Test with extension .json


Searching subdirectories:   0%|          | 0/2 [00:00<?, ?it/s]

Found 1524444 files in ./Train with extension .json


In [5]:
# Create the destination folder
if not os.path.exists(DESTINATION_FOLDER):
    os.makedirs(DESTINATION_FOLDER)

# Create test and train folders
test_data_path = os.path.join(DESTINATION_FOLDER, "Test")

if not os.path.exists(test_data_path):
    os.makedirs(test_data_path)
else:
    print(f"Folder {test_data_path} already exists")

train_data_path = os.path.join(DESTINATION_FOLDER, "Train")
if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
else:
    print(f"Folder {train_data_path} already exists")

Folder /Volumes/DATASET/Test already exists
Folder /Volumes/DATASET/Train already exists


In [6]:
def copy_file(json_path, destination_folder):
    image_path = json_path.replace(".json", ".jpg")
    
    if not os.path.exists(image_path):
        if not os.path.exists(image_path.replace(".jpg", ".jpeg")):
            print(f"Image {image_path} does not exist")
            return
        else:
            image_path = image_path.replace(".jpg", ".jpeg")

    
    if not os.path.exists(json_path):
        print(f"JSON {json_path} does not exist")
        return
    

    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    if os.path.exists(os.path.join(destination_folder, os.path.basename(json_path))):
        print(f"File {json_path} already exists in {destination_folder}")
        return
    
    if os.path.exists(os.path.join(destination_folder, os.path.basename(image_path))):
        print(f"File {image_path} already exists in {destination_folder}")
        return

    shutil.copy(json_path, destination_folder)
    shutil.copy(image_path, destination_folder)

    #delete EA files
    json_dest = os.path.join(destination_folder, os.path.basename(json_path))
    image_dest = os.path.join(destination_folder, os.path.basename(image_path))

    ea_path_json = json_dest.split("/")[:-1] + ["._" + json_dest.split("/")[-1]]
    ea_path_json = "/".join(ea_path_json)
    
    ea_path_image = image_dest.split("/")[:-1] + ["._" + image_dest.split("/")[-1]]
    ea_path_image = "/".join(ea_path_image)

    if os.path.exists(ea_path_json):
        os.remove(ea_path_json)
    
    if os.path.exists(ea_path_image):
        os.remove(ea_path_image)

In [7]:
def copy_single_batch(all_files, destination_folder):
    for json_path in tqdm(all_files, desc=f"Copying files to {destination_folder}"):
        copy_file(json_path, destination_folder)

def copy_batches(all_files, destination_folder, batch_size):
    def process_batch(batch_files, batch_folder):
        copy_single_batch(batch_files, batch_folder)

    with ThreadPoolExecutor(max_workers=16) as executor:
        futures = []
        for i in range(0, len(all_files), batch_size):
            batch_files = all_files[i:i+batch_size]
            batch_folder = os.path.join(destination_folder, f"Batch_{i//batch_size}")
            futures.append(executor.submit(process_batch, batch_files, batch_folder))

        for future in as_completed(futures):
            future.result()  # To raise any exceptions that occurred during copying

    #Check the number of files in the destination folder
    print(f"Copied {len(search_files_in_directory(destination_folder, '.json'))} json files to {destination_folder}")
    print(f"Copied {len(search_files_in_directory(destination_folder, '.jpg'))} image files to {destination_folder}")
    

In [8]:
# copy_batches(all_test_files_json, test_data_path, BATCH_SIZE)
copy_batches(all_train_files_json, train_data_path, BATCH_SIZE)

Copying files to /Volumes/DATASET/Train/Batch_767:   0%|          | 0/1000 [00:00<?, ?it/s]

Copying files to /Volumes/DATASET/Train/Batch_768:   0%|          | 0/1000 [00:00<?, ?it/s]

Copying files to /Volumes/DATASET/Train/Batch_769:   0%|          | 0/1000 [00:00<?, ?it/s]

Copying files to /Volumes/DATASET/Train/Batch_770:   0%|          | 0/1000 [00:00<?, ?it/s]

Copying files to /Volumes/DATASET/Train/Batch_771:   0%|          | 0/1000 [00:00<?, ?it/s]

...

Copying files to /Volumes/DATASET/Train/Batch_1524:   0%|          | 0/444 [00:00<?, ?it/s]

Searching subdirectories:   0%|          | 0/1525 [00:00<?, ?it/s]

Found 1524444 files in /Volumes/DATASET/Train with extension .json
Copied 1524444 json files to /Volumes/DATASET/Train


Searching subdirectories:   0%|          | 0/1525 [00:00<?, ?it/s]

Found 1464298 files in /Volumes/DATASET/Train with extension .jpg
Copied 1464298 image files to /Volumes/DATASET/Train


In [33]:
coppied_files_json = search_files_in_directory(os.path.join(DESTINATION_FOLDER,'Train'), '.json')
coppied_files_jpg = search_files_in_directory(os.path.join(DESTINATION_FOLDER,'Train'), '.jpg')


Searching subdirectories:   0%|          | 0/1525 [00:00<?, ?it/s]

Found 1524444 files in /Volumes/DATASET/Train with extension .json


Searching subdirectories:   0%|          | 0/1525 [00:00<?, ?it/s]

Found 1464298 files in /Volumes/DATASET/Train with extension .jpg


In [35]:
print(len(coppied_files_json), len(coppied_files_jpg), len(all_train_files_json))
print(f"Dif: {len(all_train_files_json) - len(coppied_files_json)}, {len(all_train_files_json) - len(coppied_files_jpg)}")

1524444 1464298 1524444
Dif: 0, 60146


In [181]:
coppied_files_jpeg = search_files_in_directory(os.path.join(DESTINATION_FOLDER,'Train'), '.jpeg')
coppied_files_jpeg

Searching subdirectories:   0%|          | 0/1525 [00:00<?, ?it/s]

Found 60146 files in /Volumes/DATASET/Train with extension .jpeg


['/Volumes/DATASET/Train/Batch_4/9b9a7fa7-40e0-4031-91ea-5386a4e3f4e0.jpeg',
 '/Volumes/DATASET/Train/Batch_4/7f1dd6cb-c206-4bd1-ac14-bc00b182d621.jpeg',
 '/Volumes/DATASET/Train/Batch_4/439f5a37-470e-41eb-b0de-a06b8f051827.jpeg',
 '/Volumes/DATASET/Train/Batch_4/3d070cec-67d4-4f3e-875b-705ac7522f4f.jpeg',
 '/Volumes/DATASET/Train/Batch_4/aafb4e58-5b84-47ca-939f-31a39cd8fea4.jpeg',
 '/Volumes/DATASET/Train/Batch_4/68280ab0-8a7a-4720-b81a-8484e7d3f599.jpeg',
 '/Volumes/DATASET/Train/Batch_4/587e1f13-ffd6-4137-b4af-d6b2332ac15c.jpeg',
 '/Volumes/DATASET/Train/Batch_4/cc5f7a59-9542-490b-bdc3-3624d416734b.jpeg',
 '/Volumes/DATASET/Train/Batch_4/d11a5a30-df64-4a3b-9f5b-80cf325acc64.jpeg',
 '/Volumes/DATASET/Train/Batch_4/d127b9c6-9f14-4b83-97ac-7e37a4d99661.jpeg',
 '/Volumes/DATASET/Train/Batch_4/54835672-ea5f-4270-ac21-25b2ff2ce078.jpeg',
 '/Volumes/DATASET/Train/Batch_4/c60f04fa-dd38-4487-bf41-206efbda248a.jpeg',
 '/Volumes/DATASET/Train/Batch_4/8f0dd4e0-4ffa-4574-be36-4aa71210be75.jpeg',

In [180]:
{f.split("/")[4] for f in coppied_files_jpeg}


{'Batch_0',
 'Batch_1',
 'Batch_10',
 'Batch_11',
 'Batch_12',
 'Batch_13',
 'Batch_14',
 'Batch_15',
 'Batch_16',
 'Batch_17',
 'Batch_18',
 'Batch_19',
 'Batch_2',
 'Batch_20',
 'Batch_21',
 'Batch_22',
 'Batch_23',
 'Batch_24',
 'Batch_25',
 'Batch_26',
 'Batch_27',
 'Batch_28',
 'Batch_29',
 'Batch_3',
 'Batch_30',
 'Batch_31',
 'Batch_32',
 'Batch_33',
 'Batch_34',
 'Batch_35',
 'Batch_36',
 'Batch_37',
 'Batch_38',
 'Batch_39',
 'Batch_4',
 'Batch_40',
 'Batch_41',
 'Batch_42',
 'Batch_43',
 'Batch_44',
 'Batch_45',
 'Batch_46',
 'Batch_47',
 'Batch_48',
 'Batch_49',
 'Batch_5',
 'Batch_50',
 'Batch_51',
 'Batch_52',
 'Batch_53',
 'Batch_54',
 'Batch_55',
 'Batch_56',
 'Batch_57',
 'Batch_58',
 'Batch_59',
 'Batch_6',
 'Batch_60',
 'Batch_7',
 'Batch_8',
 'Batch_9'}